# Research note:

We develope upon this work two approach:

* FTS online from two pictures from Imgur.

* DIV&FTS online from two pictures from Google Drive. (under development)

##### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Fast Style Transfer for Arbitrary Styles


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_arbitrary_image_stylization.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/hub/blob/master/examples/colab/tf2_arbitrary_image_stylization.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/hub/examples/colab/tf2_arbitrary_image_stylization.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub model</a>
  </td>
</table>

Based on the model code in [magenta](https://github.com/tensorflow/magenta/tree/master/magenta/models/arbitrary_image_stylization) and the publication:

[Exploring the structure of a real-time, arbitrary neural artistic stylization
network](https://arxiv.org/abs/1705.06830).
*Golnaz Ghiasi, Honglak Lee,
Manjunath Kudlur, Vincent Dumoulin, Jonathon Shlens*,
Proceedings of the British Machine Vision Conference (BMVC), 2017.


## Setup

Let's start with importing TF-2 and all relevant dependencies.

In [ ]:
import functools
import os

from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

print("TF Version: ", tf.__version__)
print("TF-Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.test.is_gpu_available())

In [ ]:
# @title Define image loading and visualization functions  { display-mode: "form" }

def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

@functools.lru_cache(maxsize=None)
def load_image(image_url, image_size=(256, 256), preserve_aspect_ratio=True):
  """Loads and preprocesses images."""
  # Cache image file locally.
  image_path = tf.keras.utils.get_file(os.path.basename(image_url)[-128:], image_url)
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img = plt.imread(image_path).astype(np.float32)[np.newaxis, ...]
  if img.max() > 1.0:
    img = img / 255.
  if len(img.shape) == 3:
    img = tf.stack([img, img, img], axis=-1)
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w  * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()


Let's get as well some images to play with.
The pictures in this first part It's from Imgur.

In [ ]:
# @title Load example images  { display-mode: "form" }

damage_image_url = 'https://i.imgur.com/V1fXu1E.jpg'  # @param {type:"string"}
style_image_url = 'https://i.imgur.com/hAWzSJX.jpg'  # @param {type:"string"}
output_image_size = 384  # @param {type:"integer"}

# The content image size can be arbitrary.
content_img_size = (output_image_size, output_image_size)
# The style prediction model was trained with image size 256 and it's the 
# recommended image size for the style image (though, other sizes work as 
# well but will lead to different results).
style_img_size = (256, 256)  # Recommended to keep it at 256.

content_image = load_image(damage_image_url, content_img_size)
style_image = load_image(style_image_url, style_img_size)
style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')
show_n([content_image, style_image], ['Content image', 'Style image'])

## Import TF-Hub module

In [ ]:
# Load TF-Hub module.

hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

The signature of this hub module for image stylization is:
```
outputs = hub_module(content_image, style_image)
stylized_image = outputs[0]
```
Where `content_image`, `style_image`, and `stylized_image` are expected to be 4-D Tensors with shapes `[batch_size, image_height, image_width, 3]`.

In the current example we provide only single images and therefore the batch dimension is 1, but one can use the same module to process more images at the same time.

The input and output values of the images should be in the range [0, 1].

The shapes of content and style image don't have to match. Output image shape
is the same as the content image shape.

## Demonstrate image stylization

In [ ]:
# Stylize content image with given style image.
# This is pretty fast within a few milliseconds on a GPU.

outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
stylized_image = outputs[0]

In [ ]:
# Visualize input images and the generated stylized image.

show_n([content_image, style_image, stylized_image], titles=['Damage', 'Style image', 'Repared'])

# DIV&FTS

## Div and conquer algorithm plus OpenCV.

Implementation of divide big pictures

https://towardsdatascience.com/images-and-masks-splitting-into-multiple-pieces-in-python-with-google-colab-2f6b2ddcb322

You need download two pictures from Imgur and upload to folder structure of this project:


*   https://i.imgur.com/V1fXu1E.jpg
*   https://i.imgur.com/hAWzSJX.jpg

Remember delete the pictures in Imput dir, and replace with other examples.

Folder structure:

*   Folder DIV&FTS\
   + Imput dir\
      - original\ V1fXu1E.jpg

      - damaged\ hAWzSJX.jpg
   + Output dir\
      - originals\ slice pictures result

      - damages\ slice pictures result

In [ ]:
import os
import sys
import shutil
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
# for impaint
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd gdrive/My Drive/Folder DIV&FTS

In [ ]:
def dir_create(path):
    """Dir create function.
        
        Create and clean a directory recrusive,
 
    """
    if (os.path.exists(path)) and (os.listdir(path) != []):
        shutil.rmtree(path)
        os.makedirs(path)
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
def crop(input_file, height, width):
    img = Image.open(input_file)
    img_width, img_height = img.size
    for i in range(img_height//height):
        for j in range(img_width//width):
            box = (j*width, i*height, (j+1)*width, (i+1)*height)
            yield img.crop(box)

In [ ]:
def split(inp_img_dir, inp_msk_dir, out_dir, height, width, 
          start_num):
    image_dir = os.path.join(out_dir, 'originals')
    mask_dir = os.path.join(out_dir, 'damages')
    dir_create(out_dir)
    dir_create(image_dir)
    dir_create(mask_dir)
    img_list = [f for f in
                os.listdir(inp_img_dir)
                if os.path.isfile(os.path.join(inp_img_dir, f))]
    file_num = 0
    for infile in img_list:
        print(infile)
        infile_path = os.path.join(inp_img_dir, infile)
        for k, piece in enumerate(crop(infile_path,
                                       height, width), start_num):
            img = Image.new('RGB', (height, width), 255)
            img.paste(piece)
            img_path = os.path.join(image_dir, 
                                    infile.split('.')[0]+ '_'
                                    + str(k).zfill(5) + '.jpg')
            img.save(img_path)

    img_list = [f for f in
                os.listdir(inp_msk_dir)
                if os.path.isfile(os.path.join(inp_msk_dir, f))]
    file_num = 0
    for infile in img_list:
        print(infile)
        infile_path = os.path.join(inp_msk_dir,
                                   infile.split('.')[0] + '.jpg')
        for k, piece in enumerate(crop(infile_path,
                                       height, width), start_num):
            msk = Image.new('RGB', (height, width), 255)
            msk.paste(piece)
            msk_path = os.path.join(mask_dir,
                                    infile.split('.')[0] + '_'
                                    + str(k).zfill(5) + '.jpg')
            msk.save(msk_path)

In [ ]:
inp_img_dir = "./input_dir/original"
inp_msk_dir = "./input_dir/damaged"
out_dir = "./output_dir"
height = 512
width = 512
start_num = 1

In [ ]:
input_images_list = glob.glob(inp_img_dir + "/*.jpg")
input_masks_list = glob.glob(inp_msk_dir + "/*.jpg")

split(inp_img_dir, inp_msk_dir, out_dir, height, width, start_num)

In [ ]:
for i, (image_path, mask_path) in enumerate(zip(input_images_list,
                                                input_masks_list)):
    fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(18, 9))
    image = mpimg.imread(image_path)
    mask = mpimg.imread(mask_path)
    ax1.set_title("Original " + str(i+1))
    ax1.imshow(image)
    ax2.imshow(mask)
    ax2.set_title("Damaged " + str(i+1))

In [ ]:
def image_part_plotter(images_list, offset,rows='', cols=''):
    """Image part plotter function.

        This function received the variables:
        + A list of images
        + Offset number (don't touch!)
        + number of rows
        + number of columns (self regulated for the length of the image list)
        
    """
    fig = plt.figure(figsize=(12, 6))
    columns = cols
    rows = rows
    # ax enables access to manipulate each of subplots
    ax = []
    for i in range(columns*rows):
        # create subplot and append to ax
        img = mpimg.imread(images_list[i+offset])
        ax.append(fig.add_subplot(rows, columns, i+1))
        ax[-1].set_title("Nº" + str(i+1))
        plt.imshow(img)
        plt.axis("off")
    plt.show() # Render the plot

In [ ]:
# This cell work together with cell 47
out_img_dir = "./output_dir/originals"
out_msk_dir = "./output_dir/damages"

In [ ]:
# We need this cell with the list of pictures and masks and iterate over the list
# and pass each pic in impairment code
output_originals_list = glob.glob(out_img_dir + "/*.jpg")
output_damages_list = glob.glob(out_msk_dir + "/*.jpg")

In [ ]:
image_part_plotter(output_originals_list,0,rows=1, cols=len(output_originals_list))
image_part_plotter(output_damages_list, 0,rows=1, cols=len(output_damages_list))

In [ ]:
def style_transfer_part_plotter(originals_list, damages_list, offset,rows='', cols=''):
    """Impaint part plotter function.

        This function received the variables:
        + A list of images
        + A list of masks
        + Offset number (don't touch!)
        + number of rows
        + number of columns (self regulated for the length of the image list)

        Then return a set of plotted and repaired by impainting pic slices.
        
    """
    fig = plt.figure(figsize=(12, 6))
    columns = cols
    rows = rows
    # ax enables access to manipulate each of subplots
    ax = []
    for i in range(columns*rows):
        
        # Stylize content image with given style image.
        # This is pretty fast within a few milliseconds on a GPU.

        outputs = hub_module(tf.constant(damages_list[i+offset]), tf.constant(originals_list[i+offset]))
        stylized_image = outputs[0]

        # create subplot and append to ax
        ax.append(fig.add_subplot(rows, columns, i+1))
        ax[-1].set_title("Nº" + str(i+1))
        plt.imshow(stylized_image)
        plt.axis("off")
    plt.show() # Render the plot

In [ ]:
style_transfer_part_plotter(output_images_list,output_masks_list,0,rows=1, cols=len(output_images_list))